In [1]:
import pandas as pd
import numpy as np
import urllib.request
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from pykospacing import Spacing
import numpy as np
from konlpy.tag import Okt
import requests
import json
import time
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation
import tensorflow as tf

warnings.filterwarnings("ignore")

In [2]:
df1 = pd.read_csv('경기도 풋살장.csv')
df2 = pd.read_csv('서울 풋살장.csv')
df3 = pd.read_csv('부산 풋살장.csv')
df4 = pd.read_csv('대구 풋살장.csv')
df5 = pd.read_csv('인천 풋살장.csv')
df6 = pd.read_csv('대전 풋살장.csv')
df7 = pd.read_csv('울산 풋살장.csv')
df8 = pd.read_csv('제주 풋살장.csv')
df9 = pd.read_csv('광주 풋살장.csv')
df10 = pd.read_csv('고양 풋살장.csv')

# 데이터 합치기
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10], ignore_index=True)
df.to_csv('주요도시 풋살장.csv', index=False, encoding='utf-8-sig')

In [3]:
# 비네이버 문자열 포함 행 모두 삭제
df = df[~df['제목'].isin(['없음', '비네이버'])]
df = df[~df['내용'].isin(['없음', '비네이버'])]
df.reset_index(drop=True, inplace=True)

In [4]:
df['제목'] = df['제목'].astype(str)
df['내용'] = df['내용'].astype(str)

In [5]:
df.drop_duplicates(subset = ['제목'], inplace=True) # 중복 제거
df.drop_duplicates(subset = ['내용'], inplace=True) 
df['제목'] = df['제목'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글을 제외하고 제거 
df['내용'] = df['내용'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") 
df['제목'] = df['제목'].str.replace('^ +', "") # 공백 제거
df['내용'] = df['내용'].str.replace('^ +', "") 
df = df.dropna(how='any') # 결측지 일괄 제거

In [6]:
df

,제목,내용
0,경기도풋살장 광주 풋살파크 오시는 길,주소 경기도 광주시 남한산성면 엄미리 풋살파크 경기도 광주시 남한산성면 ...
1,경기도 광주 풋살장 능평레포츠공원 풋살장 주차장 및 잔디 컨디션 사용후기,안녕하세요 뭅볼입니다 광주에 있는 풋살장은 이제 거의 다 가봤다고 생각했는...
2,경기도 풋살장 창업 빠른 속도로 늘어나는 축구동호회 붐,프랜차이즈 창업가맹영업리징 전문기업 이지스비즈 프창사 이 글은 이지스비즈 프창사가...
3,경기도 광주 초월읍 신월리 아카데미 풋살장 시공,시공 자재 및 장소 요약 시공장소 신월리 아카데미 풋살장 시공면적 시공잔디 킹터...
4,경기도 안양 가볼만한곳 안양새물공원 축구장 파크골프장 풋살장 테니스장 정보,축구장 파크골프장 풋살장 테니스장 정보 안양새물공원 안녕하세요 여행작가 리하파파입니...
...,...,...
7835,지상작전사령부 주임원사 챌린지코인입니다,지상작전사령부 전장의 중심 통일의 선봉 창설일 년 월 일지상작전사령부 상징명...
7837,카드뉴스 여름철 차광막 설치 야외구장 탄,여름철에도 건강하게 차광막 설치 풋살장 국민풋살앱 아이엠그라운드 추천 차광막 설치 ...
7839,리그 드림리그 개막 경기장소 시간 네이버유튜브중계 개월 애기엄마가 스트레스푸는법,이번주 당장 바로 내일 월일 지난 월입단 하고부터 기다린 리그가 개막을 합니다 근데...
7840,한강뷰 맛집 남양주 돈까스클럽 가족외식으로 딱이야,남양주돈까스클럽 남양주삼패동맛집 남양주가족외식 남양주돈까스맛집 큰애 축구훈련 끝나고...


In [7]:
shuffled_df = df.sample(frac=1, random_state=42)  # random_state는 재현성을 위한 시드 값입니다.
shuffled_df = shuffled_df.reset_index(drop=True)

In [8]:
shuffled_df.to_csv('주요도시 풋살장 셔플.csv', index=False, encoding='utf-8-sig')

## 라벨링이 된 csv 파일 불러오기

In [9]:
label_df = pd.read_csv('주요도시 풋살장 셔플 라벨.csv', encoding='utf-8-sig')

In [10]:
label_df['제목'] = label_df['제목'].astype(str)
label_df['내용'] = label_df['내용'].astype(str)

In [11]:
label_df = label_df.sample(frac=1, random_state=42)
label_df

,Label,제목,내용
5488,1,옥상 재테크 태양광 풋살장 루트톱바 변신 월 수백만원 임대료 수익 창출,옥상 재테크 태양광 풋살장 루트톱바 변신 월 수백만원 임대료 수익 창출 옥상이 금싸...
2855,1,풋살장휴공간측량,풋살장 조성 기존 노후된 풋살장을 새롭게 구상중이다 실 사용자와 관리자 감독관이 함...
2133,1,대구대구풋살장 대구 북구 에이스 풋살장,대구 북구에 위치하고 있는 에이스 풋살장입니다 지금은 태전동에서 국우동으로 이전했습...
37,1,양주 장흥 일영유원지 초원의 집 풋살장 축구장,일영유원지 장흥유원지 일영 초원의 집 일영 초원의 집 축구장 일영 초원의 집 풋살장...
2063,0,인조잔디 테니스장인조잔디 골프인조잔디 축구교실창업 풋살장창업 풋살장시공업체,안녕하세요 필드를 주름주는 사람들주식회사 스포츠 오딧세이입니다 체육시설 설계 인...
...,...,...,...
3772,1,대전축구클럽 슈퍼맨 실내축구아카데미 출동,안녕하세요 슈퍼맨 아카데미입니다 활동량이 많고 뛰어다니는 것을 좋아하며 친구들과 함...
5191,1,구로구 실내풋살장을 소개합니다,안녕하세요 실내풋살장 입니다 서울시 구로구 개봉동 매봉빌딩에 위치한 실내풋살장입니...
5226,1,대전축구교실 슈퍼맨아카데미 취미반 유소년을 위한 축구리그 개최,안녕하세요 대전축구교실 취미반 축구문화를 이끄는 대덕구 슈퍼맨축구아카데미 입니다 일...
5390,0,대전 프리즈비 모임 함께 할 횐님들을 모집합니다,대전소모임 대전운동모임 대전동호회 대전운동동호회 대전플라잉디스크 대전프리즈비 중고등...


## Preprocess titles

In [12]:
titles = label_df['제목'].tolist()

In [13]:
def preprocess(documents):
    # 불용어 불러오기
    with open('stopwords.txt', 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()

    # 텍스트 전처리
    okt = Okt()
    preprocessed = []
    for doc in documents:
        # 형태소 분석
        lemmatized = okt.nouns(doc)
        # 불용어 제거 및 토큰화
        tokens = [word for word in lemmatized if word not in stopwords]
        preprocessed.append(' '.join(tokens))

    return preprocessed

In [14]:
new_titles = preprocess(titles)

## word dictionary 생성

In [15]:
## Make a function that conduct this process
def get_vocab_dict(sents):
    word_to_idx = dict()
    idx_to_word = dict()
    index = 1
    for sent in sents:
        words = sent.split(' ')
        for word in words:
            if word not in word_to_idx.keys():
                word_to_idx[word] = index
                idx_to_word[index] = word
                index+=1
    return word_to_idx, idx_to_word

In [16]:
word_to_idx, idx_to_word = get_vocab_dict(new_titles)
word_to_idx['<pad>'] = 0
idx_to_word[0] = '<pad>'

## Integer Encoding

In [17]:
def integer_encoding(sents, word_to_idx):
    results = []
    for sent in sents:
        words = sent.split(' ')
        result = [word_to_idx[word] for word in words]
        results.append(result)
    return results

In [18]:
integer_sents = integer_encoding(new_titles, word_to_idx)

In [19]:
X_train = integer_sents[:4900]
X_test = integer_sents[4900:]
y_train = label_df['Label'][:4900].tolist()
y_test = label_df['Label'][4900:].tolist()

## Padding

In [20]:
print('제목의 최대 길이 :',max(len(i) for i in X_train))
print('제목의 평균 길이 :',sum(map(len, X_train))/len(X_train))

제목의 최대 길이 : 2950
제목의 평균 길이 : 9.712244897959184


In [21]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [22]:
max_len = 25
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 25 이하인 샘플의 비율: 98.42857142857143


In [23]:
# 총장을 25로 두고 패딩 실시
X_train_padded = pad_sequences(X_train, maxlen=max_len)
X_test_padded = pad_sequences(X_test, maxlen=max_len)

In [24]:
# model이 읽을 수 있도록 tensor 형태로 변환
X_train_tensor = tf.convert_to_tensor(X_train_padded)
X_test_tensor = tf.convert_to_tensor(X_test_padded)
y_train_tensor = tf.convert_to_tensor(y_train)
y_test_tensor = tf.convert_to_tensor(y_test)

## Vanilla LSTM Classifier 정의

In [25]:
embedding_dim = 100
hidden_units = 128

In [26]:
model = Sequential()
model.add(Embedding(len(word_to_idx), embedding_dim)) # 7386 -> 100
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

In [27]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [28]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train_tensor, y_train_tensor, epochs=15, callbacks=[es, mc], batch_size=256, validation_split=0.2)

Epoch 1/15
15/16 [===========================>..] - ETA: 0s - loss: 0.6021 - acc: 0.6445
Epoch 1: val_acc improved from -inf to 0.79796, saving model to best_model.h5
16/16 [==============================] - 3s 97ms/step - loss: 0.5993 - acc: 0.6454 - val_loss: 0.4542 - val_acc: 0.7980
Epoch 2/15
15/16 [===========================>..] - ETA: 0s - loss: 0.3483 - acc: 0.8799
Epoch 2: val_acc improved from 0.79796 to 0.92857, saving model to best_model.h5
16/16 [==============================] - 1s 64ms/step - loss: 0.3464 - acc: 0.8821 - val_loss: 0.2955 - val_acc: 0.9286
Epoch 3/15
15/16 [===========================>..] - ETA: 0s - loss: 0.2023 - acc: 0.9565
Epoch 3: val_acc did not improve from 0.92857
16/16 [==============================] - 1s 60ms/step - loss: 0.2012 - acc: 0.9566 - val_loss: 0.2577 - val_acc: 0.8786
Epoch 4/15
15/16 [===========================>..] - ETA: 0s - loss: 0.1405 - acc: 0.9602
Epoch 4: val_acc improved from 0.92857 to 0.93571, saving model to best_model.h

In [29]:
loaded_model = load_model('best_model.h5')
print("테스트 정확도: %.4f" % (loaded_model.evaluate(X_test_tensor, y_test_tensor)[1]))

65/65 [==============================] - 1s 5ms/step - loss: 0.1410 - acc: 0.9489
테스트 정확도: 0.9489


## 테스트 데이터를 분류하는 실험

In [30]:
futsal_test = pd.read_csv('풋살 테스트.csv')
futsal_answer = pd.read_csv('풋살 테스트 정답.csv')

In [31]:
new_titles = futsal_test['제목'].tolist()

In [32]:
# 전처리
new_titles = preprocess(new_titles)
# word dictionary 생성
word_to_idx, idx_to_word = get_vocab_dict(new_titles)
word_to_idx['<pad>'] = 0
idx_to_word[0] = '<pad>'
# integer encoding
integer_sents = integer_encoding(new_titles, word_to_idx)
# padding
pad_new = pad_sequences(integer_sents, maxlen = max_len)

In [33]:
# 새로운 컬럼 생성
futsal_test['Label'] = 0
futsal_test['percentage'] = 0
# 0부터 99까지의 예측값을 0.5와 대소 비교
for i in range(100):
    if (loaded_model.predict(pad_new)[i] > 0.5):
        futsal_test['Label'][i] = 1
        futsal_test['percentage'][i] = loaded_model.predict(pad_new)[i]
    else:
        futsal_test['Label'][i] = 0
        futsal_test['percentage'][i] = loaded_model.predict(pad_new)[i]

4/4 [==============================] - 0s 6ms/step


4/4 [==============================] - 0s 5ms/step


In [34]:
futsal_test['Label'].value_counts()

1    89
0    11
Name: Label, dtype: int64

In [35]:
futsal_answer['Label'] = futsal_test['Label']

In [36]:
score = 0 
for i, j in futsal_answer.iterrows():
    if j['정답'] == j['Label']:
        score += 1
print(score)

56


In [37]:
# futsal_test.to_csv('분류 테스트.csv', encoding='utf-8-sig')